In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import scienceplots
import random

from utils import combine_meta_test

In [18]:
Tag_tests = pd.read_pickle('Taguchi_test_dict.pkl') # All 32 Taguchi tests
Tag_meta = pd.read_csv('Tag_d_params.csv') # Meta data for Taguchi tests
Tag_df = combine_meta_test(Tag_meta, Tag_tests)
Tag_df = Tag_df.dropna(subset=['max_current']) # Remove any incomplete tests

In [19]:
# recreate initialization
seed = 116
Number_of_initialization_trials = 10
random.seed(seed)
choices = random.sample(range(0, 32), Number_of_initialization_trials)

In [20]:
# load meta and test data for BO designs
BO_meta_file = 'BO_designs/BO_d_params.csv' #.csv
BO_test_file = 'BayzOp_test_dict.pkl' #.pkl

BO_d_params = pd.read_csv(BO_meta_file) # all BO designs
BO_tests = pd.read_pickle(BO_test_file)
BO_df = combine_meta_test(BO_d_params, BO_tests) # combine meta and test data

In [ ]:
# add a 'Score' to Tag_df
for i, row in Tag_df.iterrows():
    if row['max_dxt'] >= 0.95:
        max_dxt = 1e12
    else:
        max_dxt = row['max_dxt']
    Tag_df.at[i, 'Score'] = row['gear_ratio'] / max_dxt

# add a 'Score' to BO_df
for i, row in BO_df.iterrows():
    if row['max_dxt'] >= 0.95:
        max_dxt = 1e12
    else:
        max_dxt = row['max_dxt']
    BO_df.at[i, 'Score'] = row['gear_ratio'] / max_dxt

In [22]:
init_df = Tag_df.iloc[choices]
# arrange by design number
init_df = init_df.sort_values(by='Design')

In [23]:
init_df['Design'] = range(0, len(init_df))

In [24]:
# combine init and BO data, init then BO:
combined_df = pd.concat([init_df, BO_df], ignore_index=True)


In [ ]:
# plot the scores by the test numbers for the two methods on the same plot
with plt.style.context(['science','ieee','no-latex']):
    plt.figure()
    sns.scatterplot(data=combined_df, x='Design', y='Score', color='r', label = 'BO')
    sns.scatterplot(data=Tag_df, x='Design', y='Score', color='b', marker='o', label='Taguchi)
    plt.show()

ValueError: Could not interpret value `Score` for `y`. An entry with this name does not appear in `data`.